In [4]:
import os
import numpy as np
import pandas as pd
import random

In [93]:
class AssetOverlap():
    def __init__(self):
        self.readFunds()
        self.readSP500()

        
    def readFunds(self):
        fundDir = os.listdir("../input/funds")
        funds = pd.DataFrame(columns=["Ticker", "ID"])
        for name in fundDir:
            fund = pd.read_csv("../input/funds/"+name, index_col=0)
            name = name.split(".csv")[0]
            
            fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
            fund = fund[["ID","Ticker",name]]
            fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no puts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
            fund = fund.dropna(axis=0, how='all')
            
            fund[name] = fund[name]/(fund[name].sum())*100 # some have over 100 percent
            
            fund.Ticker = fund.Ticker.fillna("NULL TICKER"+fund.ID)
            
            fund = fund.groupby(["Ticker","ID"]).agg({name:"sum"})# you have to groupby id and ticker or else it may merge multipletimes
            
            funds = funds.merge(fund, how='outer', on=["Ticker","ID"])
        
        
        params = {fundName:"sum" for fundName in self.getFundNames(funds)}
        params.update({"ID":lambda IDS: IDS.iloc[np.argmax([np.all([not char.isdigit() for char in ID]) for ID in IDS])] }) #first id that is all characters or first id
        funds = funds.groupby("Ticker").agg(params)
        funds = funds.reset_index()

        #print(funds.Ticker.duplicated().sum())
        
        funds = funds.fillna(0)
        self.funds = funds

    def getFundMatrix(self):
        return self.funds.loc[:,~self.funds.columns.isin(["ID","Ticker"])]
    
    def readSP500(self):
        self.sp500 = pd.read_csv("../input/s&p500.csv")
        self.sp500 = self.sp500.iloc[:,:3]
        self.sp500 = self.sp500.rename(columns={"Symbol":"Ticker", "Weight":"S&P500"})
        
    def getFundNames(self,funds):
        return [x for x in funds.columns if x not in ["ID", "Ticker"]]
    
    def makePortfolio(self,weights):
        #self.weights = self.weights.loc[self.getFundNames(self.funds)]
        columns = ["Ticker","ID"] + list(weights.index)
        self.funds = self.funds[columns]

        Portfolio = self.getFundMatrix().dot(weights)
        Portfolio = Portfolio.rename({Portfolio.columns[0]:"Portfolio"},axis=1)
        Portfolio[["Ticker","ID"]] = self.funds[["Ticker","ID"]]
        
        Portfolio = pd.merge(Portfolio, self.sp500, how='outer', on="Ticker")
        Portfolio.ID = Portfolio.ID.fillna(Portfolio.Company)
        Portfolio = Portfolio.drop("Company", axis=1)
        return Portfolio
    
    def makeAddStocksPortfolio(self,Portfolio):
        dif = Portfolio["S&P500"]-Portfolio["Portfolio"] # determine which sp500 assets have more than in the portfolio
        self.extra = Portfolio
        self.extra["Dif"] = dif
        self.extra = self.extra[self.extra.Dif>0][["Ticker","ID","Dif"]]
        self.extra = self.extra.rename(columns={"Dif":"Portfolio"})
        AddStocksPortfolio = pd.concat([Portfolio[["Ticker","ID","Portfolio"]],self.extra],axis=0)
        return AddStocksPortfolio
    
    def simpleOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        Portfolio[["Portfolio","S&P500"]] = Portfolio[["Portfolio","S&P500"]].fillna(0)
        overlap = np.minimum(np.array(Portfolio["Portfolio"]),np.array(Portfolio["S&P500"]))
        overlap = overlap.sum()
        total = Portfolio.Portfolio.sum()
        return overlap/total*100
    
    def addStocksOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return self.sp500["S&P500"].sum()/AddStocksPortfolio.Portfolio.sum()*100
    def addStocksPercentCEF(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return Portfolio.Portfolio.sum()/AddStocksPortfolio.Portfolio.sum()*100
        
    def evaluateAssetOverlap(self,weights):
        return self.simpleOverlap(weights)
        
        
        
        

In [94]:
class Discount:
    def __init__(self):
        self.readDiscount()
    def readDiscount(self):
        self.discount = pd.read_csv("../input/Discounts.csv")
        self.discount[["Discount", "52W Discount","Effective","Distribution"]] = self.discount.apply(lambda x: [x["Discount"].strip("%"),x["52W Discount"].strip("%"), x["Effective"].strip("%"), x["Distribution"].strip("%")], axis=1, result_type='expand')
        self.discount = self.discount.replace("--",0)
        self.discount.iloc[:,1:] = self.discount.iloc[:,1:].astype("float")
        
        longterm = self.discount["52W Discount"]-self.discount["Discount"] #max: a pos 52 week is pos, a neg discount is more pos
        longterm = longterm/np.max(longterm) *100# between -inf and 1
        self.discount["longterm"] = longterm
        current = -self.discount["Discount"] # max: a neg discount is pos
        current = current/np.max(-self.discount["Discount"]) * 100 #between -inf and 1
        self.discount["value"] = np.sum([0*longterm,1*current],axis=0)/1
        self.discount = self.discount.set_index("Ticker")
    
    def getWeightedDiscount(self,weights):
        percentCEF = self.addStocksPercentCEF(weights)/100
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.Discount).reshape(len(self.discount),1)
        return percentCEF * np.matmul(weights,values)[0][0]
    
    def evaluateDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.value).reshape(len(self.discount),1)
        
        return np.dot(weights,values)[0][0]
        
          

In [95]:
class TabuSet:
    def inTabuSet(self,option):
        fund1, fund2 = option
        tabuList = [adjustment.getFunds() for adjustment in self.TabuSet[:self.numTabu]]
        return [fund1,fund2] in tabuList or [fund2,fund1] in tabuList

In [96]:
class Adjustment():
    def __init__(self,fund1,fund2,value,weights):
        self.fund1 = fund1
        self.fund2 = fund2
        self.value = value
        self.weights = weights
    def getValue(self):
        return self.value
    def getWeights(self):
        return self.weights
    def getFunds(self):
        return [self.fund1, self.fund2]

In [120]:
class runModel(AssetOverlap, Discount,TabuSet):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        self.numOptions = 1
        self.numTabu = 0
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))
        
       

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        
        self.possibleFunds = list(self.weights.index)
        self.tracker = {}
        self.TabuSet = []
        #self.findInitialBest()
    
    def Evaluate(self,weights):
        asset = .7*(self.evaluateAssetOverlap(weights))
        discount = .3*self.evaluateDiscount(weights)
        return asset+discount
    
    def changeWeights(self,fund1,fund2,weights,change):
        change = min(self.getFundWeight(fund2,weights),change)
        weights.loc[fund1,"weights"] += change
        weights.loc[fund2,"weights"] -= change
        return weights
    
    def EvaluateChange(self,fund1,fund2,weights,change):
        weights = weights.copy()
        weights = self.changeWeights(fund1,fund2,weights,change)
        return self.Evaluate(weights)
    
    def getFundWeight(self,fund,weights):
        return weights.loc[fund][0]
    def findIdealChange(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)

        while self.EvaluateChange(fund1,fund2,weights,change)>control:
            #print(self.getFundWeight(fund2,weights), self.EvaluateChange(fund1,fund2,weights,change), control)
            self.changeWeights(fund1,fund2,weights,change)
            control = self.Evaluate(weights)
           
            
                
        return control, weights
    
    def makeAdjustment(self,fund1,fund2,weights,change):
        print(fund1, fund2, round(self.getFundWeight(fund1,weights),2),round(self.getFundWeight(fund2,weights),2))
        control = round(self.Evaluate(weights),2)
        
        up = round(self.EvaluateChange(fund1,fund2,weights,change),2)
        
        down = round(self.EvaluateChange(fund2,fund1,weights,change),2)
        print(round(control,2),round(up,2),round(down,2))
        if down>=control and down>up:
            fund1, fund2 = fund2, fund1
        elif control>up: #if down>control but smaller than up that wouldn't make sense so...
            for fund1,fund2 in [[fund1,fund2],[fund2,fund1]]:
                if self.getFundWeight(fund1,weights)==0 and self.getFundWeight(fund2,weights)!=0:
                    self.tracker[fund1] = self.tracker.get(fund1,0)+1
            return control,weights
        
        self.changeWeights(fund1,fund2,weights,change)
        value, weights = self.findIdealChange(fund1,fund2,weights,change)
        
        return value, weights
    def eliminateFunds(self):
        for fund,value in self.tracker.items():
            if value>=10 and fund in self.possibleFunds: 
                self.possibleFunds.remove(fund)
                
    def randomFunds(self):
        fund2 = random.choice(self.possibleFunds)
        fund1 = random.choice(self.possibleFunds)
        
        if self.getFundWeight(fund1,self.weights)==0 and self.getFundWeight(fund2,self.weights)!=0: 
            return self.randomFunds()
        return fund1, fund2
    
    def getOption(self,fund1,fund2):
        change = .1
        value, weights = self.makeAdjustment(fund1, fund2, self.weights, .01)
        return Adjustment(fund1=fund1,fund2=fund2,value=value,weights=weights)
    
            
    def runIteration(self):
        options = [self.randomFunds() for x in range(self.numOptions)]
        
        options = list(filter(lambda option: not self.inTabuSet(option),options))
        
        options = [self.getOption(fund1,fund2) for fund1,fund2 in options]
        
        if len(options)<1: return
        bestIndx = np.argmax([option.getValue() for option in options])

        option = options[bestIndx]
        self.weights = option.getWeights()
        self.TabuSet.append(option)
        self.eliminateFunds()
    
    def printEvaluation(self):
        print("Simple Overlap %.2f" % self.simpleOverlap(self.weights))
        print("Add Stocks Overlap %.2f" % self.addStocksOverlap(self.weights))
        print("Weighted Discount %.2f" % self.getWeightedDiscount(self.weights))
        
    def runModel(self):
        self.printEvaluation()
        
        for x in range(2000):
            self.runIteration()
            self.printEvaluation()
            print("\n\n")

In [121]:
obj = runModel()
obj.runModel()

Simple Overlap 35.25
Add Stocks Overlap 63.73
Weighted Discount -6.19
CET EOI 0.01 0.02
37.15 37.15 37.13
Simple Overlap 34.97
Add Stocks Overlap 63.62
Weighted Discount -6.27



BUI CLM 0.02 0.03
37.15 36.79 37.52
Simple Overlap 36.35
Add Stocks Overlap 64.18
Weighted Discount -6.16



SRV DSE 0.02 0.03
37.78 37.83 37.73
Simple Overlap 36.35
Add Stocks Overlap 64.18
Weighted Discount -6.22



BGR AEF 0.0 0.03
37.91 37.96 37.89
Simple Overlap 36.41
Add Stocks Overlap 64.20
Weighted Discount -6.24



AWP CII 0.01 0.03
37.98 37.59 38.27
Simple Overlap 36.90
Add Stocks Overlap 64.40
Weighted Discount -6.23



CLM AOD 0.04 0.03
38.27 38.17 38.37
Simple Overlap 34.94
Add Stocks Overlap 63.60
Weighted Discount -7.02



MIE CII 0.02 0.03
38.65 38.34 38.95
Simple Overlap 36.30
Add Stocks Overlap 64.16
Weighted Discount -6.92



UTF UTF 0.01 0.01
39.28 39.28 39.28
Simple Overlap 36.30
Add Stocks Overlap 64.16
Weighted Discount -6.92



ETJ CPZ 0.03 0.01
39.28 39.32 39.23
Simple Overlap 36.80
Ad

Simple Overlap 49.51
Add Stocks Overlap 70.10
Weighted Discount -7.79



BDJ BSTZ 0.0 0.0
48.95 48.95 48.95
Simple Overlap 49.51
Add Stocks Overlap 70.10
Weighted Discount -7.79



CPZ BMEZ 0.04 0.01
48.95 49.17 48.68
Simple Overlap 49.64
Add Stocks Overlap 70.16
Weighted Discount -7.89



CEM BSTZ 0.03 0.0
49.21 49.21 49.1
Simple Overlap 49.64
Add Stocks Overlap 70.16
Weighted Discount -7.89



CRF BCX 0.03 0.0
49.21 49.21 49.21
Simple Overlap 49.64
Add Stocks Overlap 70.16
Weighted Discount -7.89



EOS GLQ 0.09 0.0
49.21 49.21 48.96
Simple Overlap 49.64
Add Stocks Overlap 70.16
Weighted Discount -7.89



RQI CTR 0.0 0.0
49.21 49.21 49.21
Simple Overlap 49.64
Add Stocks Overlap 70.16
Weighted Discount -7.89



BOE ETJ 0.02 0.1
49.21 49.25 49.09
Simple Overlap 49.39
Add Stocks Overlap 70.04
Weighted Discount -8.00



AWP BSTZ 0.0 0.0
49.25 49.25 49.25
Simple Overlap 49.39
Add Stocks Overlap 70.04
Weighted Discount -8.00



EOI NFJ 0.01 0.01
49.25 49.26 49.23
Simple Overlap 49.58
Add S

53.22 53.22 53.22
Simple Overlap 55.65
Add Stocks Overlap 73.25
Weighted Discount -8.13



ETJ AEF 0.09 0.0
53.22 53.22 53.12
Simple Overlap 55.65
Add Stocks Overlap 73.25
Weighted Discount -8.13



GRF ETJ 0.05 0.09
53.22 53.32 53.07
Simple Overlap 54.03
Add Stocks Overlap 72.39
Weighted Discount -8.82



EMO BGR 0.0 0.0
53.48 53.48 53.48
Simple Overlap 54.03
Add Stocks Overlap 72.39
Weighted Discount -8.82



IAF CEM 0.0 0.0
53.48 53.48 53.48
Simple Overlap 54.03
Add Stocks Overlap 72.39
Weighted Discount -8.82



CLM BDJ 0.0 0.0
53.48 53.48 53.48
Simple Overlap 54.03
Add Stocks Overlap 72.39
Weighted Discount -8.82



AGD EOI 0.01 0.07
53.48 53.36 53.55
Simple Overlap 54.24
Add Stocks Overlap 72.50
Weighted Discount -8.79



CTR AGD 0.0 0.0
53.55 53.55 53.55
Simple Overlap 54.24
Add Stocks Overlap 72.50
Weighted Discount -8.79



CLM BGY 0.0 0.0
53.55 53.55 53.55
Simple Overlap 54.24
Add Stocks Overlap 72.50
Weighted Discount -8.79



DPG CEN 0.0 0.0
53.55 53.55 53.55
Simple Overlap

Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



DNP AOD 0.0 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



BSTZ BMEZ 0.0 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



SZC CEN 0.13 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



AOD JEQ 0.0 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



AEF ASA 0.0 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



UTF AWP 0.0 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



GRF BCX 0.07 0.0
56.8 56.8 56.7
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



RNP CRF 0.0 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Discount -9.32



BUI AIO 0.0 0.0
56.8 56.8 56.8
Simple Overlap 58.22
Add Stocks Overlap 74.66
Weighted Disco

59.71 59.71 59.71
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



AIO RNP 0.0 0.0
59.71 59.71 59.71
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



ADX DSE 0.38 0.0
59.71 59.71 59.49
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



CTR RFI 0.0 0.0
59.71 59.71 59.71
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



RFI ASA 0.0 0.0
59.71 59.71 59.71
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



BIF AOD 0.02 0.01
59.71 59.68 59.6
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



GRF BCX 0.02 0.0
59.71 59.71 59.38
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



DPG SRV 0.0 0.0
59.71 59.71 59.71
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



IAF RFI 0.0 0.0
59.71 59.71 59.71
Simple Overlap 58.66
Add Stocks Overlap 74.90
Weighted Discount -10.87



IAF BDJ 0.0 0.0
59.71 59.71 59.71
Simple 

...

Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



CRF RNP 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



CPZ RNP 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



UTF RQI 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



EOS IAF 0.01 0.0
61.73 61.73 61.48
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



CLM AOD 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



UTF EMO 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



EXD DPG 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



EOS BOE 0.01 0.01
61.73 61.72 61.71
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.79



BMEZ STK 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.49
Add

Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



BDJ GRF 0.05 0.05
61.73 61.67 61.7
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



BDJ MIE 0.05 0.0
61.73 61.73 61.57
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



UTF AWP 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



BGR DSE 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



BCX CEM 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



CEM DPG 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



AEF CEN 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



CPZ STK 0.0 0.0
61.73 61.73 61.73
Simple Overlap 62.39
Add Stocks Overlap 77.06
Weighted Discount -10.82



GRF BGY 0.05 0.0
61.73 61.73 61.49
Simple Overlap 62.39
Add 

61.74 61.74 61.32
Simple Overlap 62.33
Add Stocks Overlap 77.02
Weighted Discount -10.85



EXD DNP 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.33
Add Stocks Overlap 77.02
Weighted Discount -10.85



EOS BOE 0.01 0.02
61.74 61.74 61.68
Simple Overlap 62.61
Add Stocks Overlap 77.18
Weighted Discount -10.76



AOD DPG 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.61
Add Stocks Overlap 77.18
Weighted Discount -10.76



SZC IGR 0.04 0.0
61.74 61.74 61.64
Simple Overlap 62.61
Add Stocks Overlap 77.18
Weighted Discount -10.76



BDJ CPZ 0.05 0.0
61.74 61.74 61.69
Simple Overlap 62.61
Add Stocks Overlap 77.18
Weighted Discount -10.76



CTR BCX 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.61
Add Stocks Overlap 77.18
Weighted Discount -10.76



BIF DSE 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.61
Add Stocks Overlap 77.18
Weighted Discount -10.76



BGR CEN 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.61
Add Stocks Overlap 77.18
Weighted Discount -10.76



BGY AOD 0.0 0.0
61.74 61.74 61.74
Simple

Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.80



CRF IAF 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.80



AIO AGD 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.80



PEO RFI 0.04 0.0
61.74 61.74 61.33
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.80



BUI RNP 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.80



BGR DPG 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.80



DNP CLM 0.0 0.0
61.74 61.74 61.74
Simple Overlap 62.49
Add Stocks Overlap 77.11
Weighted Discount -10.80



CII BDJ 0.01 0.05
61.74 61.75 61.71
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



RFI RNP 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



AWP AOD 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add 

Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



BDJ CEM 0.04 0.0
61.75 61.75 61.55
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



NFJ CTR 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



CHN DPG 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



MIE CRF 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



BMEZ JEQ 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



AGD AEF 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



IAF CEN 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



CLM JEQ 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add Stocks Overlap 77.15
Weighted Discount -10.78



CEN SRV 0.0 0.0
61.75 61.75 61.75
Simple Overlap 62.55
Add S

In [119]:
obj.tracker

{}

In [112]:
obj.weights.loc["ADX"] -= .01
obj.weights.loc["CET"] += .01

In [113]:
obj.getWeightedDiscount(obj.weights)

-10.299278975298398

In [114]:
obj.Evaluate(obj.weights)

61.405338517223356

In [115]:
obj.simpleOverlap(obj.weights)

63.361420262465074

In [100]:
obj.weights.sort_values(by="weights",ascending=False)

,weights
ADX,0.540504
CET,0.139214
EOI,0.098075
SZC,0.047942
BDJ,0.040385
PEO,0.036078
GRF,0.032504
EOS,0.030000
CII,0.020925
NFJ,0.014373


In [23]:
list(obj.weights.index[obj.weights.weights!=0])

['ADX', 'SZC', 'CII', 'GRF', 'EOI', 'BDJ', 'EOS', 'CET', 'BOE', 'PEO']

In [147]:
Discount().discount

,Effective,Distribution,Discount,52W Discount,longterm,value
Ticker,,,,,,
ADX,0.00,20.55,-13.27,-13.74,-4.304029,36.468780
AEF,7.90,2.64,-12.89,-14.09,-10.989011,33.155048
AGD,6.94,7.51,-12.70,-13.11,-3.754579,35.023824
AIO,2.95,5.05,-6.89,-10.25,-30.769231,9.423681
AOD,0.09,7.84,-13.81,-13.28,4.853480,41.063699
ASA,0.00,0.09,-15.96,-15.45,4.670330,47.143752
AWP,7.68,9.28,-12.22,-13.72,-13.736264,30.325558
BCX,1.20,6.63,-14.22,-15.29,-9.798535,37.350788
BDJ,0.27,7.19,-9.53,-9.34,1.739927,27.800770


In [114]:
def t(ab):
    ab = ab[:]
   
    ab.iloc[0] = [1,3]
    print(ab)
    
    
ki = pd.DataFrame([[1,2]])
t(ki)
ki
#i set the list copy to that and then though I could efdit it after word. but psych apparently that was me editing it and seeting list to something else doesn;t work othersie

   0  1
0  1  3


,0,1
0,1,3
